# Deploying FastAPI Applications with Docker and Docker Swarm


In this tutorial, we'll explore how to containerize a **FastAPI** application using **Docker** and deploy it using **Docker Swarm**. We'll cover the fundamental concepts of Docker and Docker Swarm, how to set up your environment, and demonstrate how to use these tools to build, deploy, and scale your FastAPI application. By the end of this tutorial, you'll be able to create a highly available and scalable API service.

## Table of Contents

1. [Introduction](#1-introduction)
2. [Prerequisites](#2-prerequisites)
3. [Understanding Docker and Docker Swarm](#3-understanding-docker-and-docker-swarm)
   - [3.1. What is Docker?](#31-what-is-docker)
   - [3.2. What is Docker Swarm?](#32-what-is-docker-swarm)
4. [Setting Up the Environment](#4-setting-up-the-environment)
   - [4.1. Installing Docker](#41-installing-docker)
   - [4.2. Setting Up a Swarm Cluster](#42-setting-up-a-swarm-cluster)
5. [Creating a Sample FastAPI Application](#5-creating-a-sample-fastapi-application)
6. [Containerizing the Application with Docker](#6-containerizing-the-application-with-docker)
   - [6.1. Writing a Dockerfile](#61-writing-a-dockerfile)
   - [6.2. Building the Docker Image](#62-building-the-docker-image)
   - [6.3. Running the Docker Container Locally](#63-running-the-docker-container-locally)
7. [Deploying the Application with Docker Swarm](#7-deploying-the-application-with-docker-swarm)
   - [7.1. Creating a Docker Swarm Service](#71-creating-a-docker-swarm-service)
   - [7.2. Scaling the Service](#72-scaling-the-service)
   - [7.3. Updating the Service](#73-updating-the-service)
8. [Using Docker Compose for Swarm Deployment](#8-using-docker-compose-for-swarm-deployment)
   - [8.1. Writing a docker-compose.yml File](#81-writing-a-docker-composeyml-file)
   - [8.2. Deploying the Stack](#82-deploying-the-stack)
9. [Networking and Load Balancing](#9-networking-and-load-balancing)
   - [9.1. Overlay Networks](#91-overlay-networks)
   - [9.2. Ingress Load Balancing](#92-ingress-load-balancing)
10. [Persistent Data with Volumes](#10-persistent-data-with-volumes)
11. [Health Checks and Rolling Updates](#11-health-checks-and-rolling-updates)
    - [11.1. Implementing Health Checks](#111-implementing-health-checks)
    - [11.2. Performing Rolling Updates](#112-performing-rolling-updates)
12. [Logging and Monitoring](#12-logging-and-monitoring)
13. [Conclusion](#13-conclusion)
14. [References](#14-references)

## 1. Introduction

**Docker** is a platform for developing, shipping, and running applications in containers. **Docker Swarm** is a clustering and scheduling tool for Docker containers that turns a pool of Docker hosts into a single, virtual host. By combining FastAPI with Docker and Docker Swarm, you can build scalable, distributed, and highly available API services.

In this tutorial, we'll:

- Understand the basics of Docker and Docker Swarm.
- Containerize a FastAPI application using Docker.
- Deploy the application using Docker Swarm.
- Explore scaling, updating, and managing the application using Docker Swarm features.
- Learn about networking, load balancing, and persistent data management.

## 2. Prerequisites

Before we begin, ensure you have the following:

- **Python 3.7+** installed.
- Basic knowledge of **Python**, **FastAPI**, and **Docker**.
- Familiarity with command-line operations.
- **Docker Engine** installed on your machine.
- (Optional) Multiple machines or virtual machines to simulate a Swarm cluster.

## 3. Understanding Docker and Docker Swarm

### 3.1. What is Docker?

**Docker** is an open-source platform designed to automate the deployment of applications inside lightweight, portable containers. Containers bundle an application's code with its dependencies, ensuring consistency across environments.

### 3.2. What is Docker Swarm?

**Docker Swarm** is Docker's native orchestration tool that allows you to create and manage a cluster of Docker engines, called a swarm. It enables you to deploy services across multiple nodes, providing high availability and scalability.

**Key Features:**

- **Scalability**: Easily scale up or down the number of container instances.
- **Load Balancing**: Automatically distributes traffic across containers.
- **Rolling Updates**: Update services without downtime.
- **Service Discovery**: Built-in DNS for service name resolution.

## 4. Setting Up the Environment

### 4.1. Installing Docker

#### On macOS and Windows

Download and install **Docker Desktop** from the [official website](https://www.docker.com/products/docker-desktop).

#### On Linux

Install Docker Engine by following the instructions for your distribution on the [Docker Docs](https://docs.docker.com/engine/install/).

#### Verify Installation

```bash
docker --version
```

### 4.2. Setting Up a Swarm Cluster

For testing purposes, you can initialize a swarm on a single machine. In production, you would use multiple nodes.

#### Initialize the Swarm

```bash
docker swarm init --advertise-addr <MANAGER-IP>
```

- **<MANAGER-IP>**: Replace with your machine's IP address.

#### Add Worker Nodes (Optional)

If you have additional machines, you can join them to the swarm as worker nodes.

On the manager node, run:

```bash
docker swarm join-token worker
```

This command outputs a `docker swarm join` command with a token. Run that command on the worker nodes to join the swarm.

## 5. Creating a Sample FastAPI Application

Create a new project directory and set up a virtual environment.

```bash
mkdir fastapi-docker-swarm
cd fastapi-docker-swarm
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate
```

Install FastAPI and Uvicorn:

```bash
pip install fastapi uvicorn
```

Create `app/main.py`:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def read_root():
    return {"message": "Hello, Docker Swarm!"}

@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}
```

Test the application locally:

```bash
uvicorn app.main:app --reload
```

Visit `http://localhost:8000` to see the output.

## 6. Containerizing the Application with Docker

### 6.1. Writing a Dockerfile

Create a file named `Dockerfile` in the project directory:

```dockerfile
# Use the official Python image as the base image
FROM python:3.9-slim

# Set the working directory
WORKDIR /app

# Copy the requirements file
COPY requirements.txt .

# Install the dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code
COPY . .

# Expose the port
EXPOSE 80

# Command to run the application
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]
```

Create `requirements.txt`:

```text
fastapi
uvicorn[standard]
```

### 6.2. Building the Docker Image

Build the Docker image:

```bash
docker build -t fastapi-docker-swarm:latest .
```

Verify the image is built:

```bash
docker images
```

### 6.3. Running the Docker Container Locally

Run the Docker container:

```bash
docker run -d --name fastapi-app -p 8000:80 fastapi-docker-swarm:latest
```

Visit `http://localhost:8000` to confirm it's working.

## 7. Deploying the Application with Docker Swarm

### 7.1. Creating a Docker Swarm Service

Deploy the application as a service in the swarm:

```bash
docker service create \
  --name fastapi_service \
  --publish published=8000,target=80 \
  --replicas 3 \
  fastapi-docker-swarm:latest
```

**Explanation:**

- **--name fastapi_service**: Names the service.
- **--publish published=8000,target=80**: Maps port 80 inside the container to port 8000 on the host.
- **--replicas 3**: Runs 3 instances of the service.
- **fastapi-docker-swarm:latest**: Specifies the image to use.

### 7.2. Scaling the Service

Scale the service up or down:

```bash
docker service scale fastapi_service=5
```

Check the status of the service:

```bash
docker service ls
docker service ps fastapi_service
```

### 7.3. Updating the Service

Make changes to your application code (e.g., update the message in `app/main.py`):

```python
return {"message": "Hello, Docker Swarm! Updated version."}
```

Rebuild the Docker image:

```bash
docker build -t fastapi-docker-swarm:latest .
```

Update the service to use the new image:

```bash
docker service update --image fastapi-docker-swarm:latest fastapi_service
```

Docker Swarm will perform a rolling update, gradually replacing old containers with new ones.

## 8. Using Docker Compose for Swarm Deployment

Docker Compose can be used to define and run multi-container applications. With Docker Swarm, you can use Docker Compose files to deploy services.

### 8.1. Writing a `docker-compose.yml` File

Create a `docker-compose.yml` file:

```yaml
version: '3.8'

services:
  fastapi:
    image: fastapi-docker-swarm:latest
    deploy:
      replicas: 3
      update_config:
        parallelism: 1
        delay: 10s
      restart_policy:
        condition: on-failure
    ports:
      - "8000:80"
    networks:
      - webnet

networks:
  webnet:
```

**Explanation:**

- **deploy** section: Contains deployment configurations for Swarm.
- **networks**: Defines an overlay network.

### 8.2. Deploying the Stack

Deploy the stack:

```bash
docker stack deploy -c docker-compose.yml fastapi_stack
```

Check the status:

```bash
docker stack ls
docker stack services fastapi_stack
```

## 9. Networking and Load Balancing

### 9.1. Overlay Networks

Docker Swarm uses overlay networks to enable communication between services across different nodes.

- The `webnet` network in the Compose file is an overlay network.
- Containers can communicate using service names.

### 9.2. Ingress Load Balancing

Docker Swarm provides built-in load balancing:

- Requests to the published port are distributed among service replicas.
- You can test load balancing by making multiple requests and observing which container handles each request.


## 10. Persistent Data with Volumes

If your application requires persistent storage, you can use Docker volumes.

Modify the Compose file:

```yaml
volumes:
  - data-volume:/data

volumes:
  data-volume:
```

Mount the volume in the service:

```yaml
services:
  fastapi:
    # ... existing configurations ...
    volumes:
      - data-volume:/data
```

## 11. Health Checks and Rolling Updates

### 11.1. Implementing Health Checks

Add a health check to the service to ensure that containers are functioning correctly.

Update the Compose file:

```yaml
services:
  fastapi:
    # ... existing configurations ...
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:80/"]
      interval: 30s
      timeout: 10s
      retries: 3
```

### 11.2. Performing Rolling Updates

When updating the service, Docker Swarm can perform rolling updates with zero downtime.

```bash
docker service update --image fastapi-docker-swarm:latest fastapi_service
```

You can control the update behavior with parameters like `--update-parallelism` and `--update-delay`.

## 12. Logging and Monitoring

Use Docker's logging drivers to collect logs.

View service logs:

```bash
docker service logs fastapi_service
```

For advanced monitoring, consider using tools like **Prometheus** and **Grafana**, or Docker's built-in **Docker Metrics**.

## 13. Conclusion

In this tutorial, we've:

- Learnt about Docker and Docker Swarm and their key features.
- Containerized a FastAPI application using Docker.
- Deployed the application to a Docker Swarm cluster.
- Explored scaling, updating, and managing the application using Docker Swarm features.
- Understood networking, load balancing, and persistent data management in Docker Swarm.
- Implemented health checks and performed rolling updates.

By deploying your FastAPI application with Docker and Docker Swarm, you can ensure scalability, high availability, and efficient resource utilization.

## 14. References

- [Docker Official Documentation](https://docs.docker.com/)
- [Docker Swarm Documentation](https://docs.docker.com/engine/swarm/)
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Docker Compose File Reference](https://docs.docker.com/compose/compose-file/)
- [Docker Networking](https://docs.docker.com/network/)
- [Docker Stack Deploy](https://docs.docker.com/engine/reference/commandline/stack_deploy/)